<a href="https://colab.research.google.com/github/anilozdemir/EchoVPR/blob/main/notebooks/example_train_single_ESN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get EchoVPR from GitHub

In [1]:
!git clone https://github.com/anilozdemir/EchoVPR.git

Cloning into 'EchoVPR'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 51 (delta 11), reused 34 (delta 6), pack-reused 0
Unpacking objects: 100% (51/51), done.


## Install `echovpr` module

In [2]:
%cd EchoVPR/src

/content/EchoVPR/src


In [3]:
!python setup.py develop

running develop
running egg_info
creating echovpr.egg-info
writing echovpr.egg-info/PKG-INFO
writing dependency_links to echovpr.egg-info/dependency_links.txt
writing top-level names to echovpr.egg-info/top_level.txt
writing manifest file 'echovpr.egg-info/SOURCES.txt'
writing manifest file 'echovpr.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/echovpr.egg-link (link to .)
Adding echovpr 1.0 to easy-install.pth file

Installed /content/EchoVPR/src
Processing dependencies for echovpr==1.0
Finished processing dependencies for echovpr==1.0


# Train ESN and ESN+SpaRCe

In [4]:
import torch

from echovpr.datasets    import getHiddenRepr, DataSets, Tolerance
from echovpr.networks    import singleESN, getSparsity
from echovpr.experiments import SingleESN_Exp, getValidationIndices

## Get NetVLAD Hidden Representation and Validation Indices

In [5]:
ds  = 'GardensPoint'
tol = Tolerance[ds]

# Get NetVLAD Hidden Representation
hiddenReprTrain, hiddenReprTest = getHiddenRepr('GardensPoint')
# Get Input and Output Size; First element of shape: (number of images == number of classes == nOutput); Second element:  size of hidden representations
nOutput, nInput = hiddenReprTrain.shape 
# Get Validation Indices
TestInd, ValidationInd = getValidationIndices(1, nOutput)

# ESN Training

In [6]:
nRes     = 1000
nCon     = 10
nTrial   = 10
nEpoch   = 50
nBatch   = 5
lR       = 0.01
gamma    = 0.0003 
alpha    = 0.68

In [7]:
_ESN_model_ = lambda randSeed: singleESN(nInput, nOutput, nReservoir=nRes, randomSeed = randSeed, device='cpu', useReadout = False,
                                     sparsity = getSparsity(nCon, nRes), alpha = alpha, gamma = gamma, rho = 0.99)

In [8]:
model   = _ESN_model_(0)
exp     = SingleESN_Exp(model, hiddenReprTrain, hiddenReprTest, TestInd, ValidationInd , tol)
results = exp.train_esn(nEpoch=nEpoch, lR=lR, nBatch=nBatch, returnData=True, returnDataAll=False)
results['AccTest'][-1].mean()

0.8299999999999998

# ESN+SpaRCe Training

In [9]:
nRes     = 1000
nCon     = 10
nTrial   = 10
nEpoch   = 50
nBatch   = 5
lR       = 0.01
gamma    = 0.0003 
alpha    = 0.74
quantile = 0.4 
lrDiv    = 10

In [10]:
_SPARCE_model_ = lambda randSeed: singleESN(nInput, nOutput, nReservoir = nRes, randomSeed = randSeed, device='cpu', useReadout = False,
                                     sparsity = getSparsity(nCon, nRes), alpha = alpha, gamma = gamma, rho = 0.99)

In [11]:
model   = _SPARCE_model_(0)
exp     = SingleESN_Exp(model, hiddenReprTrain, hiddenReprTest, TestInd, ValidationInd , tol)
results = exp.train_sparce(nEpoch=nEpoch, lR=lR, nBatch=nBatch, quantile=quantile, lr_divide_factor=lrDiv, returnData=True, returnDataAll=False)
results['AccTest'][-1].mean()

0.74